# OpenAI Thread API

## Overview

This is a brief end-to-end example of how to use the more recent Assistants/Threads OpenAI API; the [docs]() 
are reasonably good, but only provide piecemeal examples of how to use the Python API, and I wanted to
have a feel for the full end-to-end functionality.

Here, we will create one `Assistant`, then a `Thread` (OpenAI's evolution of the `Chat Completions`) and then
execute a `Run` of the `Thread` using the given `Assistant`, to provide the LLM output.

Obviously, in a "real" application, this would be execute in an actual conversation thread, iteratively sending
to the LLM subsequent user prompts, until the problem is solved.

The full code is in [this GitHub repository](https://github.com/massenz/hugging) along with other code samples
using the [Hugging Face](https://hugging.dev) API.

In [76]:
from common import read_env
from openai import OpenAI
import time

# These are only needed for type annotations
from openai.resources.beta.assistants import Assistant
from openai.types.beta.thread import Thread
from openai.types.beta.threads.run import Run


### Authentication

Recently, OpenAI has introduced the concept of a `Project` (make sure you are using
the most recent version of the `openai` package, `1.25.1`):

```
└─( pip freeze | grep -i openai
openai==1.25.1
```
Assistants are scoped by-project, so this is important; omitting the `project` keyword arg
will make use of the `default project` (whatever was in the org account prior to the
projects' introduction).

More info [here](https://platform.openai.com/docs/api-reference/authentication).

In [94]:
env = read_env()
client = OpenAI(api_key=env['oai_token'], 
                organization=env['oai_org_id'], 
                project=env['oai_project_id'],
         )

OAI_MODEL = env.get('oai_model') or 'gpt-4-turbo'

# Assistants

An `Assistant` receives specific instructions, a personality, a specialization and is used to customize the responses from the LLM (what is usually called `Prompt Engineering`).

In the example below, we create a `Go Developer` assistant, then create two utility functions to retrieve all the assistants in the Project, and to reverse-map their names to their `id`.

**Note**
> We have "pinned" the model to the `OAI_MODEL` value above, but this can be dynamically changed.
> Also, make sure that you use at least `gpt-4-turbo` which is up-to-date to Dec 2023
> (more info [here](https://platform.openai.com/docs/models/gpt-4-turbo-and-gpt-4)).

In [95]:
def get_assistants() -> dict[str, str]:
    """ Retrieves the list of assistants, scoped by org/project."""
    my_assistants = client.beta.assistants.list(
        order="desc",
        limit="20",
    )
    assts = {}
    for a in my_assistants.data:
        assts[a.name] = a.id
    return assts


def get_asst_id(name: str) -> str:
    """Given an Assistant's name, will return its ID."""
    return get_assistants().get(name, None)


def get_asst_by_name(name: str) -> Assistant:
    """Retrieves an Assistant by name"""
    aid = get_asst_id(name)
    return client.beta.assistants.retrieve(assistant_id=aid)


def new_assistant(name: str, instructions: str) -> Assistant:
    # First, check that an assistant with that name
    # does not already exist; OpenAI will otherwise create a
    # new one with the same name, which is probably not what the 
    # user expected.
    aid = get_asst_id(name)
    if aid is not None:
        print(f"WARN: Assistant {name} already exists, updating instructions.")
        return client.beta.assistants.update(
            assistant_id=aid,
            name=name,
            instructions=instructions,
            model=OAI_MODEL,
        )
    return client.beta.assistants.create(
                name=name,
                instructions=instructions,
                tools=[{"type": "code_interpreter"}],
                model=OAI_MODEL,
            )

# Threads

A `Thread` is exactly what the name says: a conversation thread, with a sequence of messages, between the `user` and the `assistant`; it is used in a `Run` (see below) to drive the LLM.

In [96]:
def new_thread() -> Thread:
    return client.beta.threads.create()
    
def add_msg_to_thread(thread: Thread, message: str) -> None:
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )

# Runs

A `Run` embodies the execution of a `Thread` for an `Assistant`; see [here](https://platform.openai.com/docs/assistants/how-it-works/run-lifecycle) for a full description of a `Run`'s lifecycle.

Once the `Run` is in the `"completed"` state...
> You can also continue the conversation by adding more user Messages to the Thread and creating another Run.


In [97]:
def new_run(thread: Thread, asst_name: str) -> Run:
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=get_asst_id(asst_name),
    )

def wait_on_run(run, thread, timeout: int = 120, interval: float = 2) -> bool:
    count = 0
    retries = timeout / interval
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )    
        count += 1
        if count > retries:
            # We have exceeded the timeout, we need to first 
            # cancel the run to avoid incurring further costs.
            client.beta.threads.runs.cancel(
                thread_id=thread.id,
                run_id=run.id,
            )
            return False
        time.sleep(0.5)
    return run.status == "completed"

def get_response(thread: Thread) -> str:
    """Simplified approach to retrieve just the latest message from a 'completed' run.

    Generally, there may be several assistant's messages in a Run; however, in the simple
    case here, we can assume the LLM provided its response in a single message, the last
    in the queue.
    """
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        order="asc",
    )
    if len(messages.data) == 0:
        raise ValueError("No messages in Thread")
    return messages.data[-1].content[0].text.value

# Using the LLM

We are now ready to request our `Assistant` to provide us with guidance and help; this is typically done in a loop, here we just show a couple of iterations.

In [98]:
name = "Go Dev"

instructions = """You are a dedicated GoLang developer,
and have detailed knowledge of the Go OpenAI library
called github.com/sashabaranov/go-openai.
You will provide your answers formatted in Markdown,
with appropriate code examples.
The code should be complete and ready to be compiled: do not
provide just fragments, but the full code for functions and types.
"""

content = """
Using the `go-openai` library in Go, please show me a function 
that creates an OpenAI Assistant, and associate it to
a Thread;
then create a function that takes that Thread, and an arbitrary
user content, and creates a message and adds it to the Thread;
finally, a third function should execute the Run with the Thread,
and return a string with the response from OpenAI GPT-4.

Please, do not make things up; if you don't know how to do something,
please feel free to ask the question, and we'll take it from there.
"""

# 0. We need an assistant
new_assistant(name, instructions)

# 1. Get a Thread, and append a message to it
thread = new_thread()
add_msg_to_thread(thread, content)

# 2. Get a new Run, and associate it with our Thread
#    We will use the Assistant (Go Dev) we created before.
run = new_run(thread=thread, asst_name=name)

# 3. We then ask GPT for advice
if wait_on_run(run, thread):
    response = get_response(thread)
    print(f"{name} says:\n{response}")
else:
    print(f"We failed! Status: {run.status}, {run.incomplete_details}") 

WARN: Assistant Go Dev already exists, updating instructions.
Go Dev says:
Here's how you can use the `go-openai` package in Go to interact with the OpenAI API, particularly focusing on creating an assistant, managing a thread, and processing messages with OpenAI GPT-4.

### Setup

1. **Create an Assistant**: An assistant needs to be created, which can be attached to subsequent threads for message handling.
2. **Manage a Thread**: A thread is like a conversation or dialog which holds the messages.
3. **Execute the Run**: This actually sends the message to OpenAI and gets the response which is processed accordingly.

First, please ensure that you have the `go-openai` library installed:

```bash
go get github.com/sashabaranov/go-openai
```

Here's an example illustrating the entire process:

```go
package main

import (
	"context"
	"fmt"
	"github.com/sashabaranov/go-openai/openai"
)

// Initializes a new OpenAI client with your API key
func initClient(apiKey string) *openai.Client {
	cli

In [99]:
# We can append another message to the Thread and continue the conversation.
content = """
We should be able to read the API_KEY and an ORG_ID and PRJ_ID
from a configuration YAML file (for now, assume it is called `config.yaml`)
with the following structure:

```yaml
openai:
    api_key: ts_12345
    org_id: org_998743424
    prj_id: prj_afad4456

model: gpt-4-turbo
```

Please add a `GetConfig(name string) (Config, error)` function that does that
and invoke it from the `main()` function.
"""

add_msg_to_thread(thread, content)
run = new_run(thread=thread, asst_name=name)
if wait_on_run(run, thread):
    response = get_response(thread)
    print(f"{name} says:\n{response}")
else:
    print(f"We failed! Status: {run.status}, {run.incomplete_details}") 

Go Dev says:
Certainly! I'll provide the function `GetConfig` which will read the configuration details from a `config.yaml` file using the popular `gopkg.in/yaml.v2` package for YAML parsing in Go. Below is the full code including adjustments to read the config file and retrieve necessary settings.

Firstly, you will need to install the YAML package:

```bash
go get gopkg.in/yaml.v2
```

### Full Code Revised with Configuration Reader

```go
package main

import (
	"context"
	"fmt"
	"github.com/sashabaranov/go-openai/openai"
	"gopkg.in/yaml.v2"
	"io/ioutil"
	"log"
)

type Config struct {
	OpenAI struct {
		APIKey string `yaml:"api_key"`
		OrgID  string `yaml:"org_id"`
		PrjID  string `yaml:"prj_id"`
	} `yaml:"openai"`
	Model string `yaml:"model"`
}

// GetConfig reads from a YAML file and unmarshals into Config struct
func GetConfig(filename string) (Config, error) {
	var config Config

	data, err := ioutil.ReadFile(filename)
	if err != nil {
		return Config{}, err
	}

	err = yaml.Unm

### Put all the pieces together

Hopefully, by now it should be pretty obvious how all the moving parts can be put together (ideally, in appropriate classes) and the whole process ran in a loop, until the user declares the task complete.

The Jupyter Notebook can be found [here](https://github.com/massenz/hugging/blob/main/OpenAI%20API%20Examples.ipynb).